## 모델 I / O

In [ ]:
from langchain import PromptTemplate

template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘"

prompt = PromptTemplate(input_variables=["product"], template=template)

prompt.format(product="카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘'

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

llm1 = ChatOpenAI(temperature=0, model_name="gpt-4o")

prompt = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"

print(llm1.invoke(prompt).content)


# invoke는 content 속성에 답변을 담고있고,
# 토큰 사용량, 모델 이름등의 메타데이터와 기타 추가정보를 담고있다.

강아지입니다.


In [23]:
from langchain.llms import HuggingFaceHub
import os
from dotenv import load_dotenv

load_dotenv()

hug_api = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm2 = HuggingFaceHub(
    repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_new_tokens": 100}
)

prompt = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
completion = llm2.invoke(prompt, max_new_tokens=100, stream=False)

print(completion)

c:\ProgramData\anaconda3\envs\seSac_env\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?
미물은 키우고 있는 미물은?
미물은 키우고 있는 미물은?
미물은 키우고 있는 미물은?



### ModelLaboratory로 LLM모델 생성 결과 비교하기

In [ ]:
from langchain.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([llm1, llm2])
model_lab.compare("대한민국의 가을은 몇 월 부터 몇 월 까지야?")

## Output_Parser를 통해 출력 결과 포멧 지정하기

In [34]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, max_tokens=2048, model="gpt-4o")

output_parser = CommaSeparatedListOutputParser()  # 파서 초기화
format_instructions = output_parser.get_format_instructions()  # 출력 형식 지정

prompt = PromptTemplate(
    template="7개의 팀을 보여줘 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

query = "한국의 야구팀은?"

# 출력 결과 생성
output = llm.invoke(input=prompt.format(subject=query))
print(output)
print("==" * 50, "\n")
# 출력에 대한 포멧 변경
parsered_result = output_parser.parse(output.content)
print(parsered_result)

content='두산 베어스, LG 트윈스, 키움 히어로즈, SSG 랜더스, NC 다이노스, 롯데 자이언츠, 삼성 라이온즈' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 48, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_6dd05565ef', 'finish_reason': 'stop', 'logprobs': None} id='run-658a956c-1d7e-482c-9da9-43b1115ac735-0'

['두산 베어스', 'LG 트윈스', '키움 히어로즈', 'SSG 랜더스', 'NC 다이노스', '롯데 자이언츠', '삼성 라이온즈']


## 데이터 연결 (PDF)

In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./The_Adventures_of_Tom_Sawyer.pdf")
document = loader.load()
document[5].page_content[:5000]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he \ndidn’t like working. He liked playing and having \nadventures. One Friday, he didn’t go to school—he went \nto the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you \ndidn’t go to school today. Tomorrow you’re going to work \nfor me. You can paint the fence.” \nSaturday morning, Tom was not happy, but he started to \npaint the fence. His friend Jim was in the street. \nTom asked him, “Do you want to paint?” \nJim said, “No, I can’t. I’m going to get water.” \nThen Ben came to Tom’s house. He watched Tom and \nsaid, “I’m going to swim today. You can’t swim because \nyou’re working.” \nTom said, “This isn’t work. I like painting.” \n“Can I paint, too?” Ben asked. \n“No, you can’t,” Tom answered. “Aunt Polly asked me \nbecause I’m a very good painter.” \nBen said, “I’m a good pai

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()  # 임베딩 모듈 초기화
db = FAISS.from_documents(document, embeddings)

C:\Users\user\AppData\Local\Temp\ipykernel_12168\3427618908.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()  # 임베딩 모듈 초기화


In [3]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

text = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
text_embedding = embeddings.embed_query(text)
print(text_embedding[:10])

[-0.0028138665231909497, -0.020238374776143958, -0.012738939981790699, -0.016539159741274387, -0.02259930910767389, 0.028709960370719174, -0.02787668932061184, 0.0063410627904474165, -0.012025609306402632, 0.009797242220737239]


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

text = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
embedded_text = embeddings.embed_query(text)
print(embedded_text[:5])

C:\Users\user\AppData\Local\Temp\ipykernel_12168\3798662067.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


[0.013278793543577194, 0.07225918024778366, 0.0926310271024704, -0.00397957768291235, 0.0015618291217833757]


In [8]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

from langchain.chains import RetrievalQA

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "마을 무덤에 있던 남자를 죽인 사람은 누구고 어떻게 죽였어?"
result = qa({"query": query})
print(result["result"])

마을 무덤에서 의사를 죽인 사람은 인디언 조예요. 그는 칼로 의사를 죽였어요.


### Chain

In [ ]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(temperature=0, model="gpt-4o")
prompt = PromptTemplate(
    input_varidables=["country"], template="{country}의 수도는 어디야?"
)

chain = LLMChain(llm=llm, prompt=prompt)  # 프롬프트와 모델을 체인으로 연결
chain.run("캐나다")

'캐나다의 수도는 오타와입니다.'

In [12]:
# 프롬프트 1 정의
prompt1 = PromptTemplate(
    input_variabels=["sentence"],
    template="다음 문장을 한글로 번역하세요 \n\n {sentence}",
)

# 번역체인 1에 대한 모델
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="stranslation")

# 프롬프트 2 정의
prompt2 = PromptTemplate.from_template(
    "다음 문장을 한 문장으로 요약하세요. \n\n {stranslation}"
)

# 요약체인 2 에 대한 모델
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="summary")

from langchain.chains import SequentialChain

all_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["sentence"],
    output_variables=["stranslation", "summary"],
)

sentence = """
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""
all_chain(sentence)

{'sentence': '\nOne limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.\nFor this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.\n',
 'stranslation': 'LLM의 한 가지 제한점은 특정 문서나 이메일에 대한 접근과 같은 맥락 정보가 부족하다는 것입니다. 이를 해결하기 위해 LLM에 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다. 이를 위해 먼저 문서 로더를 사용하여 외부 데이터를 로드해야 합니다. LangChain은 PDF, 이메일, 웹사이트, YouTube 동영상 등 다양한 유형의 문서에 대한 다양한 로더를 제공합니다.',
 'summary': 'LLM의 맥락 정보 부족 문제를 해결하기 위해 LangChain은 PDF, 이메일, 웹사이트, YouTube 동영상 등 다양한 외부 데이터를 로드할 수 있는 문서 로더를 제공합니다.'}

## 메모리

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, name="gpt-4o")

from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="진희는 강아지를 한마리 키우고 있습니다.")
conversation.predict(input="영수는 고양이를 두마리 키우고 있습니다.")
conversation.predict(input="진희와 영수가 키우는 동물은 총 몇마리?")

C:\Users\user\AppData\Local\Temp\ipykernel_12168\2768668027.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, verbose=True)
c:\ProgramData\anaconda3\envs\seSac_env\lib\site-packages\pydantic\main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 진희는 강아지를 한마리 키우고 있습니다.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 진희는 강아지를 한마리 키우고 있습니다.
AI: 진희 씨는 강아지를 키우고 있군요! 강아지의 이름이 무엇인가요?
Human: 영수는 고양이를 두마리 키우고 있습니다.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of

'진희 씨는 강아지 한 마리를 키우고, 영수 씨는 고양이 두 마리를 키우고 있으니, 총 세 마리의 동물을 키우고 있습니다.'

### Agent

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

from langchain.agents import load_tools, initialize_agent, AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    description="계산이 필요할 때 사용",
    verbose=True,
)

agent.run("아이브 장원영이 태어난 해는? 2025년 현재의 나이는?")

C:\Users\user\AppData\Local\Temp\ipykernel_12168\1331107071.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,




> Entering new AgentExecutor chain...
To answer the question, I need to find out the birth year of Jang Wonyoung and then calculate her age in 2025.

Action: wikipedia
Action Input: Jang Wonyoung

Observation: Page: Jang Won-young
Summary: Jang Won-young (Korean: 장원영; born August 31, 2004), also known mononymously as Wonyoung, is a South Korean singer. She is a member of the South Korean girl group Ive under Starship Entertainment and former member of project girl group Iz*One, having finished first in Mnet's girl group survival reality television show Produce 48 in 2018.

Page: Sunghoon (singer, born 2002)
Summary: Park Sung-hoon (Korean: 박성훈; born December 8, 2002), known mononymously as Sunghoon, is a South Korean singer and former figure skater. He competed as a figure skater from 2010 to early 2020; while simultaneously being a K-pop trainee since 2018. He retired from the sport and debuted as a member of the South Korean boy band Enhypen in November 2020. Sunghoon is the 2016–2

'Jang Wonyoung was born in 2004, and she will be 21 years old in 2025.'